# Example 0: Integrating in Python
Ok, so you need to do an integral. You are using Python, so you decide to use quad() from scipy.integrate. The thing is, your integral ends up taking forever. How can we speed this up? Well, the answer is to write your function in C and call it from in Python. Let's see how.

In [1]:
#Import everything we need
import numpy as np
import time
from scipy.integrate import quad

Our integrand is $\cos(x)/x^3$ evaluated from $\pi$ to $2\pi$. Yes, I know, this can be done analytically. This is just for demonstrative purposes. Pretend it was more complicated. The answer to this integral is approximately -0.0152115.

In [2]:
def integrand(x):
    return np.cos(x)/(x*x*x)

We now have our integrand in Python-function form. Let's time how long it takes to do this integral one hundred thousand times.

In [3]:
start = time.time()
for i in range(100000):
    answer = quad(integrand, np.pi, 2*np.pi)
end = time.time()
print "Got answer: ", answer
print "Time with Python integrand: ", end - start

Got answer:  (-0.015211452880075338, 2.903565890448967e-16)
Time with Python integrand:  3.89219403267


# Example 1: Basic use of ctypes
I also have the integrand written in C in the file integrand.c. The text of the file is:

```c
#include<math.h>

double integrand(int n, double x){
  return cos(x)/(x*x*x);
}
```

To compile the file, we can use the makefile by simply calling $make. Inside the makefile we have the following:

```
integrand:
	gcc -shared -o integrand.so -fPIC integrand.c
```

The two flags that must be turned on to make a *shared object (.so)* file are -shared and -fPIC. Now we can get this .so object and its functions into our Python program.

In [4]:
import ctypes
lib = ctypes.cdll.LoadLibrary("integrand.so")
c_integrand = lib.integrand
c_integrand.argtypes = (ctypes.c_int, ctypes.c_double)
c_integrand.restype  = ctypes.c_double

In [5]:
start = time.time()
for i in range(100000):
    answer = quad(c_integrand, np.pi, 2*np.pi)
end = time.time()
print "Got answer: ", answer
print "Time with ctypes integrand: ", end - start

Got answer:  (-0.015211452880075338, 2.903565890448967e-16)
Time with ctypes integrand:  1.57994008064


**NOTE**: an integrand being used for quad needs the signature as we have written it: an integer followed by a double, with a double as the return type. This is for compatability with nquad(), which I'll get to in another notebook.

Also, the exact difference in performance at this point depends on what version of gcc/numpy/scipy you have. The optimization was more pronounced for my laptop (factor of >2 increase) which had a more recent version of gcc than calvin.

# Example 2: Extra arguments
Let's say that our integrand was no $\cos(ax)/x^b$ and we didn't know what $a$ and $b$ were ahead of time. They should now be arguments for our function. Here is how to handle this with Python.

In [6]:
def integrand2(x, a, b):
    return np.cos(a*x)/x**b

print quad(integrand2, np.pi, 2*np.pi, args=(1.0, 3.0))

(-0.015211452880075338, 2.903565890448967e-16)


I have now added an extra function to the integrand.c file:

```c
double integrand2(int n, double args[n]){
  double x = args[0];
  double a = args[1];
  double b = args[2];
  return cos(a*x)/pow(x,b);
}
```

Now, the integration variable and the arguments are all passed through the array $x$. The length of this array is specified by $n$. Spoiler: this gives us some insight into how this and nquad are implemented. In nquad, the x[n] array is used for all of the integration variables and the arguments. This seems not well thought out, compared to other integrators.

Let's use ctypes again to get at this function and integrate it.

In [7]:
c_integrand2 = lib.integrand2
c_integrand2.argtypes = (ctypes.c_int, ctypes.c_double)
c_integrand2.restype  = ctypes.c_double

print quad(c_integrand2, np.pi, 2*np.pi, args=(1.0, 3.0))

(-0.015211452880075338, 2.903565890448967e-16)


Boom. We get the same answer whether we use Python or C.

# Example 3: Splines (aka look up tables)
Usually in our group we need to do integrals over the power spectrum $P(k)$ or the mass function $\frac{dn}{dM}(M)$ and there is no analytic form of part of or all of our integrand. In this case we use a spline, but a spline is a complicated object in Python, which cannot be passed directly to C without serious fenagling with Cython (which I don't know how to use). Our options are:
1. Pass in $P$ and $k$, and create a spline each time we call the integrand (bad - slow).
2. Use an integrator in C, and write a wrapper that calls this, and then return the result.

We will use option 2. This is under development.